# 0. Import needed Libraries

In [ ]:
# the 3 main libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
# librairies to do nicer plots...
from matplotlib.patches import Rectangle    # only to draw a rectangle

# 1. Get data:


### Select the SST product you whant to use

In [ ]:
#sstproduct = 'erssst'
sstproduct = 'oisst'

### read SST and get land-sea mask

In [ ]:
if sstproduct == "erssst":
    
    data = xr.tutorial.open_dataset('ersstv5')
    data = data.sel(lat=slice(60., -60.))          # get rid of data out of 60S-60N
    data = data.sel(time=slice('1970','2021'))     # select years between 1970 and 2021
    mask = data.sst.isel(time=0)
    mask.data = np.where(np.isnan(mask.data), 0., 1.)
    
elif sstproduct == "oisst":
    
    data = xr.open_dataset('sst.mnmean.nc')
    data = data.sel(time=slice('1982','2022'))     # select years between 1982 and 2022
    data_mask = xr.open_dataset('lsmask.nc')
    mask = data_mask.mask                          # get the DataArray from the Dataset
    mask = mask.squeeze()                          # get rid of the time dimension in the variable mask
    
else:

    print("wrong name of SST product")

In [ ]:
data

### Reduce memory footprint

In [ ]:
data = data.sel(lat=slice(45, -45))      # get rid of data out of 45S-45N
mask = mask.sel(lat=slice(45, -45))      # get rid of data out of 45S-45N

### Compute Weights

In [ ]:
weights = np.cos( np.deg2rad(mask.lat) )
weights = mask*weights
weights.plot()

# 2. linear trend:

### Long term mean trend 
[linear regression](http://xarray.pydata.org/en/stable/computation.html?highlight=polyval#fitting-polynomials) with [polyfit](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.polyfit.html?highlight=polyfit) and [polyval](http://xarray.pydata.org/en/stable/generated/xarray.polyval.html?highlight=polyval) methonds

In [ ]:
linfit = data.sst.polyfit('time', 1)               # compute linear regression coefficients
linfit.polyfit_coefficients

In [ ]:
slope = linfit.polyfit_coefficients.isel(degree=0)   # Polynomial coefficients, highest power first
ns_century = 1.e9*3600.*24.*365.*100.                # scaling: nono-seconds to century
(slope*ns_century).where(mask == 1.).plot(vmin=-5,vmax=5,cmap='RdBu_r')
plt.title('Trend Slope in deg/century')

### Detrend SST

In [ ]:
trend = xr.polyval(coord=data.time, coeffs=linfit.polyfit_coefficients)   # SST trend
sst_detrend = data.sst - trend.values + data.sst.mean(dim='time')         # detrended SST

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(6,3), constrained_layout=True)
a = data.sst.groupby('time.year').mean(dim='time')      # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(ax=axes[0], label="org")
a = trend.groupby('time.year').mean(dim='time')         # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(ax=axes[0], label="trend")
axes[0].legend()
axes[0].set_title('With trend')
#
a = sst_detrend.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(ax=axes[1], label="detrend")
axes[1].legend()
axes[1].set_title('Without trend')

### Detrended interannual anomaly

In [ ]:
sstbymth = sst_detrend.groupby("time.month")
mthclim = sstbymth.mean("time")       # detrended climatological months (seasonal cycle)
sstanom = sstbymth - mthclim          # detrended interannual annomaly
sstanom = sstanom.rename('sstanom')   # change variable name

Interannual anomaly (with/without trend)

In [ ]:
a = data.sst.weighted(weights).mean(dim=('lon','lat'))
clim = a.groupby("time.month").mean("time")   # climatological months (seasonal cycle)
anom = a.groupby("time.month") - clim         # interannual annomaly
anom.plot(label="org")
#
sstanom.weighted(weights).mean(dim=('lon','lat')).plot(label="detrended")
plt.legend()

# 3. SST Regression onto Nino3.4:

### Interannual standard deviation
with [nino3.4](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/) box (5S-5N and 170-120W).

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5), constrained_layout=True)
sst_detrend.std(dim='time').where(mask == 1.).plot(ax=axes[0],vmin=0,vmax=5,cmap='YlGnBu')
axes[0].set_title('SST detrend STD')
sstanom.std(dim='time').where(mask == 1.).plot(ax=axes[1],vmin=0,vmax=2,cmap='YlGnBu')
axes[1].set_title('SST detrend + anom STD')
# overplot nino3.4 rectangle (170W-120W, 5S-5N)
ax = plt.gca()                                                              # Get the current reference
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='r',fill=False)   # Create a Rectangle patch
ax.add_patch(rect)                                                          # Add the patch to the Axes

### Equatorial hovmoller of the SST interannual anomaly

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10, 10))
wgteq = weights.sel(lat=slice(2.5,-2.5), lon=slice(100.5, 285.5))
#
ssteq = sst_detrend.sel(lat=slice(2.5,-2.5), lon=slice(100.5, 285.5))
ssteq.weighted(wgteq).mean(dim='lat').plot(ax=axes[0])
#
ssteq = sstanom.sel(lat=slice(2.5,-2.5), lon=slice(100.5, 285.5))
ssteq.weighted(wgteq).mean(dim='lat').plot(ax=axes[1])

### Compute nino3.4 (5S-5N and 170-120W) index

In [ ]:
weights_nino34 = weights.sel(lon=slice(360-169.5,360-120.5), lat=slice(4.5,-4.5))
nino34_index = sstanom.weighted(weights_nino34).mean(dim=('lon','lat'))
nino34_index = nino34_index.rename('nino34')   # change variable name
nino34_index.plot()

### Monthly std of nino3.4 index

In [ ]:
nino34_index.groupby('time.month').std(dim='time').plot.step(where='mid')

### Nino/Nina dissymmetry

In [ ]:
print(nino34_index.where(nino34_index > 0.).std().data)
print(nino34_index.where(nino34_index < 0.).std().data)

### SST anomaly maps for some Nino Events (DJF)

In [ ]:
nino34_index.where(nino34_index > 1.5, drop=True).time

In [ ]:
fig, axes = plt.subplots(7,1, figsize=(6, 16), constrained_layout=True)
dataarray = sstanom.sel(time=slice('1982-12','1983-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[0], vmin=-2, vmax=2, cmap='RdBu_r')
axes[0].set_title('82-83')
dataarray = sstanom.sel(time=slice('1987-12','1988-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[1], vmin=-2, vmax=2, cmap='RdBu_r')
axes[1].set_title('87-88')
dataarray = sstanom.sel(time=slice('1991-12','1992-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[2], vmin=-2, vmax=2, cmap='RdBu_r')
axes[2].set_title('91-92')
dataarray = sstanom.sel(time=slice('1997-12','1998-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[3], vmin=-2, vmax=2, cmap='RdBu_r')
axes[3].set_title('97-98')
dataarray = sstanom.sel(time=slice('2002-12','2003-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[4], vmin=-2, vmax=2, cmap='RdBu_r')
axes[4].set_title('02-03')
dataarray = sstanom.sel(time=slice('2009-12','2010-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[5], vmin=-2, vmax=2, cmap='RdBu_r')
axes[5].set_title('09-10')
dataarray = sstanom.sel(time=slice('2015-12','2016-02')).mean(dim='time').where(mask == 1.)
dataarray.plot(ax=axes[6], vmin=-2, vmax=2, cmap='RdBu_r')
axes[6].set_title('15-16')

### SST anomalies regressed onto Nino3.4 SST

In [ ]:
sstanom.coords["time"] = (("time"), nino34_index.data)  # redefine 'time' coordinates with nino34 time series
linfit = sstanom.polyfit('time', 1)                     # compute the regression with this new "time" 
sstanom.coords["time"] = (("time"), data.time.data)     # put back original time

sstreg = linfit.polyfit_coefficients.isel(degree=0)
sstcor = xr.corr(sstanom, nino34_index, dim='time')                      

#  create a 2 pannels figure
fig, axes = plt.subplots(1,2,figsize=(14, 4))
sstreg.where(mask == 1.).plot(ax=axes[0])                                # plot the regression coefficient
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[0].add_patch(rect)                                                     # overlay 
axes[0].set_title('Regression coefficient')
sstcor.where(mask == 1.).plot(ax=axes[1])                                # plot the correlation
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[1].add_patch(rect)                                                     # overlay 
axes[1].set_title('Correlation')
axes[1].set_xlabel('lon')
axes[1].set_ylabel('lat')